<a href="https://colab.research.google.com/github/msaantonova/Accent-Classification_Speech-technologies/blob/main/Accent_recognition_project_SS25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

Repository from the paper

"This repository provides all the necessary tools to perform accent identification from speech recordings with SpeechBrain toolkit! The system uses a model fine-tuned on the CommonAccent dataset in English (21 accents). The provided system can recognize the following 21 accents of English from short speech recordings"

In [ ]:
!git clone https://github.com/lgspeech/accent-recog-slt2022

Cloning into 'accent-recog-slt2022'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 527 (delta 182), reused 155 (delta 154), pack-reused 323 (from 1)
Receiving objects: 100% (527/527), 2.64 MiB | 10.97 MiB/s, done.
Resolving deltas: 100% (342/342), done.


In [ ]:
%cd /content/accent-recog-slt2022
!python -m pip install -r requirements.txt
# you can ignore any errors for now

/content/accent-recog-slt2022
Ignoring SoundFile: markers 'sys_platform == "win32"' don't match your environment
  Using cached datasets-2.8.0-py3-none-any.whl.metadata (19 kB)
  Using cached pre_commit-4.2.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached scipy-1.6.1.tar.gz (27.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Ignored the following yanked versions: 2.0.0
ERROR: Ignored the following versions that require a different python version: 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10; 1.7.2 Requires-Python >=3.7,<3.11; 1.7.3 Requires-Python >=3.7,<3.11; 1.8.0 Requires-Python >=3.8,<3.11; 1.8.0rc1 Requires-Python >=3.8,<3.11; 1.8.0rc2 Requires-Python >=3.8,<3.11; 1.8.0rc3 Requires-Python >=3.8,<3.11; 1.8.0rc4 Requires-Python >=3.8,<3.11; 1.8.1 Requires-Python >=3.8,<3.11
ERROR: Coul

In [ ]:
!pip install -q datasets==3.0.0

In [ ]:
# change working directory:
%cd CommonAccent/

/content/accent-recog-slt2022/CommonAccent


# Download the data (or stream) and create the list of filtered items

You can either use streaming, or save the files locally. you can check at common voice, how big the corpus for each language is.
If you use streaming, it  will take a few hours.

The aim of it is to create lists of all audios that have marked accent in the metadata, so you can choose which ones of these you want for classification.

In [ ]:
#this part descrbes the dataset index and the language
!python download_data_hf.py --language "it" data/cv_11/

python3: can't open file '/content/download_data_hf.py': [Errno 2] No such file or directory


# Prepare the filtered list to be used in this model --> create csv files in correct form

In [ ]:
!pip install -q speechbrain==0.5.13 \
    transformers==4.25.1 \
    librosa==0.9.2 \
    ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.0/499.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━ 175.5/363.4 MB 110.8 MB/s eta 0:00:02
ERROR: Operation cancelled by user
^C


KeyboardInterrupt: 

In [ ]:
!python common_accent_prepare.py --language "it" data/cv_11 data/

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.io.wavfile import write
from datasets import load_dataset

data_path = "/content/accent-recog-slt2022/CommonAccent/data/"
# Load CSV
csv_path_train = data_path + "train.csv"
df_train = pd.read_csv(csv_path_train)
file_list_train = list(set(df_train['wav']))

csv_path_test = data_path + "test.csv"
df_test = pd.read_csv(csv_path_test)
file_list_test = list(set(df_test['wav']))

csv_path_dev = data_path + "dev.csv"
df_dev = pd.read_csv(csv_path_dev)
file_list_dev = list(set(df_dev['wav']))

# Set save directory
save_dir = data_path + "wav_files"
os.makedirs(save_dir, exist_ok=True)

# Load dataset in streaming mode
language = "it"  # or whatever language code you're using
cv_19_train = load_dataset("fsicoli/common_voice_19_0", language, streaming=True, split="train", trust_remote_code=True)
print(next(iter(cv_19_train)))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Reading metadata...: 171388it [00:07, 23662.65it/s]


{'client_id': 'c14f21cacf2b7701ad0bead0dd1b31ec9d3a1557708e446de55e98b4b470cf31072c82543e5ba518c5c187a91868878a4e32727054a3dd94f9df41c9a13d8c62', 'path': 'it_train_0/common_voice_it_17415777.mp3', 'audio': {'path': 'it_train_0/common_voice_it_17415777.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -1.61342832e-05, -1.57291943e-05, -1.16234423e-05]), 'sampling_rate': 48000}, 'sentence': "Il marchese aveva già moglie in quell'epoca?", 'up_votes': 3, 'down_votes': 0, 'age': '', 'gender': '', 'accent': '', 'locale': 'it', 'segment': '', 'variant': ''}


In [ ]:
cv_19_dev = load_dataset("fsicoli/common_voice_19_0", language, streaming=False, split="validation", trust_remote_code=True)
cv_19_test = load_dataset("fsicoli/common_voice_19_0", language, streaming=False, split="test", trust_remote_code=True)

In [ ]:
import pandas as pd
# Count entries per accent
accent_counts = df_train['accent'].value_counts()

# Print the result
print(accent_counts)

# select randomly max 300 samples for each accent that has more than 300 samples


accent
TENDENTE AL SICULO MA NON MARCATO    1801
BASILICATA TRENTINO                  1801
VENETO                               1393
MERIDIONALE                           134
EMILIANO                               91
Name: count, dtype: int64


# This code adjusts the audio paths and can be used for balancing the data:

In [ ]:
import pandas as pd

# add /content/accent-recog-slt2022/CommonAccent/data_small to each "wav" in the csv .../train.csv
save_dir_colab = data_path
# "content/accent-recog-slt2022/CommonAccent/data_small/"
df_train['wav'] = df_train['wav'].apply(lambda x: os.path.join(save_dir_colab, x))
df_test['wav'] = df_test['wav'].apply(lambda x: os.path.join(save_dir_colab, x))
df_dev['wav'] = df_dev['wav'].apply(lambda x: os.path.join(save_dir_colab, x))
df_train.to_csv(csv_path_train, index=False)

# show all available accents
accent_counts = df_train['accent'].value_counts()

# Print the result
print(accent_counts)

# select randomly max 300 samples for each accent
df_train2 = df_train.groupby('accent').apply(lambda x: x.sample(min(len(x), 300), random_state=1)).reset_index(drop=True)
df_dev2 = df_dev.groupby('accent').apply(lambda x: x.sample(min(len(x), 50), random_state=1)).reset_index(drop=True)
df_test2 = df_test.groupby('accent').apply(lambda x: x.sample(min(len(x), 50), random_state=1)).reset_index(drop=True)
# remove accents with less than 300 samples
df_train2 = df_train2.groupby('accent').filter(lambda x: len(x) >= 300)
# get names of remaining accents
remaining_accents = df_train2['accent'].unique()
# filter dev and test sets to keep only the remaining accents
df_dev2 = df_dev2[df_dev2['accent'].isin(remaining_accents)]
df_test2 = df_test2[df_test2['accent'].isin(remaining_accents)]
# reduce to 50 samples for dev and test sets

# print stats of the new dataframe
accent_counts2_train = df_train2['accent'].value_counts()
accent_counts2_dev = df_dev2['accent'].value_counts()
accent_counts2_test = df_test2['accent'].value_counts()
# Print the result
print(accent_counts2_train)
print(accent_counts2_dev)
print(accent_counts2_test)

# save csv again with these samples
os.makedirs('/content/accent-recog-slt2022/CommonAccent/data_small', exist_ok=True)
df_train2.to_csv('/content/accent-recog-slt2022/CommonAccent/data_small/train.csv', index=False)
df_dev2.to_csv('/content/accent-recog-slt2022/CommonAccent/data_small/dev.csv', index=False)
df_test2.to_csv('/content/accent-recog-slt2022/CommonAccent/data_small/test.csv', index=False)
# download files to disk
!zip -r /content/accent-recog-slt2022/CommonAccent/data_small.zip /content/accent-recog-slt2022/CommonAccent/data_small
# download
from google.colab import files
files.download("/content/accent-recog-slt2022/CommonAccent/data_small.zip")

accent
TENDENTE AL SICULO MA NON MARCATO    1801
BASILICATA TRENTINO                  1801
VENETO                               1393
MERIDIONALE                           134
EMILIANO                               91
Name: count, dtype: int64
accent
BASILICATA TRENTINO                  300
TENDENTE AL SICULO MA NON MARCATO    300
VENETO                               300
Name: count, dtype: int64
accent
BASILICATA TRENTINO                  50
TENDENTE AL SICULO MA NON MARCATO    50
VENETO                               50
Name: count, dtype: int64
accent
BASILICATA TRENTINO                  50
TENDENTE AL SICULO MA NON MARCATO    50
VENETO                               50
Name: count, dtype: int64


<ipython-input-20-f3c15a7b6ddf>:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train2 = df_train.groupby('accent').apply(lambda x: x.sample(min(len(x), 300), random_state=1)).reset_index(drop=True)
<ipython-input-20-f3c15a7b6ddf>:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_dev2 = df_dev.groupby('accent').apply(lambda x: x.sample(min(len(x), 50), random_state=1)).reset_index(drop=True)
<ipytho

In [ ]:
# save audios locally (takes around 27min for italian)
for item1 in cv_19_train:
    # Check if the current file path is in the CSV file list
    # print(item['path'])
    # adjust the following line to your working directory
    filename = (item1['path'].split('/')[-2] + '/' + item1['path'].split('/')[-1])
    if filename in file_list_train or filename in file_list_test or filename in file_list_dev:
      # Decode audio
      audio_array = item1['audio']['array']  # Audio waveform
      sampling_rate = item1['audio']['sampling_rate']  # Sampling rate

      # Create the filename
      file_name = os.path.join(save_dir, filename)
      os.makedirs(os.path.dirname(file_name), exist_ok=True)

      # Save the audio as a WAV file
      write(file_name, sampling_rate, audio_array.astype(np.float32))  # Save as float32 for WAV compatibility


In [ ]:
# takes 2 min for it dev, test
for item1 in cv_19_dev:
    filename = (item1['path'].split('/')[-2] + '/' + item1['path'].split('/')[-1])
    if filename in file_list_train or filename in file_list_test or filename in file_list_dev:
      # Decode audio
      audio_array = item1['audio']['array']  # Audio waveform
      sampling_rate = item1['audio']['sampling_rate']  # Sampling rate

      # Create the filename
      file_name = os.path.join(save_dir, filename)
      os.makedirs(os.path.dirname(file_name), exist_ok=True)

      # Save the audio as a WAV file
      write(file_name, sampling_rate, audio_array.astype(np.float32))  # Save as float32 for WAV compatibility

In [ ]:
for item1 in cv_19_test:
    filename = (item1['path'].split('/')[-2] + '/' + item1['path'].split('/')[-1])
    if filename in file_list_train or filename in file_list_test or filename in file_list_dev:
      # Decode audio
      audio_array = item1['audio']['array']  # Audio waveform
      sampling_rate = item1['audio']['sampling_rate']  # Sampling rate

      # Create the filename
      file_name = os.path.join(save_dir, filename)
      os.makedirs(os.path.dirname(file_name), exist_ok=True)

      # Save the audio as a WAV file
      write(file_name, sampling_rate, audio_array.astype(np.float32))  # Save as float32 for WAV compatibility

After this step I would suggest to copy the .csv files and also the wav files to your gdrive so that you can load them from there next time.
Next time (if you saved the wav files in your gdrive) you can copy it back to colab and can continue from there.

In [ ]:
# !mkdir -p /content/accent-recog-slt2022/CommonAccent/data_small
# !cp -R /content/drive/MyDrive/.../data/wav /content/accent-recog-slt2022/CommonAccent/data_small/wav_files

In [ ]:
!cp -R /content/accent-recog-slt2022/CommonAccent/data_small/wav_files /content/drive/MyDrive/.../data/wav

In [ ]:
import soundfile as sf
import librosa
import os

# resample all files to 16khz
base_dir = '/content/accent-recog-slt2022/CommonAccent/data_small/wav_files/wav_files/'

# Walk through all subdirectories
for root, dirs, files in os.walk(base_dir):
    for filename in files:
        if filename.endswith('.mp3'):
            audio_file = os.path.join(root, filename)
            try:
                # Load with librosa to ensure mono and 16kHz resampling
                data, _ = librosa.load(audio_file, sr=16000, mono=True)
                # Output path with .wav extension
                new_path = audio_file.replace('.mp3', '.wav')
                # Save as 16kHz mono .wav
                sf.write(new_path, data, 16000)
                # print(f"Converted and saved: {new_path}")
            except Exception as e:
                print(f"Error processing {audio_file}: {e}")


# Install dependencies for training the model
You should not get any errors here

In [ ]:
!pip install -q pandas==2.2.2 \
    huggingface_hub>=0.7.0 \
    hyperpyyaml==0.0.1 \
    joblib \
    numpy==2.0.2 \
    packaging \
    pre-commit==2.3.0 \
    sentencepiece>=0.1.91 \
    SoundFile \
    tqdm

In [ ]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# You will probably have to adjust your audiopaths in the csv files:

In [ ]:
!cp /content/accent-recog-slt2022/CommonAccent/data_small/train.csv /content/accent-recog-slt2022/CommonAccent/data_small/train_orig.csv
!cp /content/accent-recog-slt2022/CommonAccent/data_small/dev.csv /content/accent-recog-slt2022/CommonAccent/data_small/dev_orig.csv
!cp /content/accent-recog-slt2022/CommonAccent/data_small/test.csv /content/accent-recog-slt2022/CommonAccent/data_small/test_orig.csv

In [ ]:
import os
import csv
import soundfile as sf
import librosa
import numpy as np

def update_csv_with_full_path_and_duration(
    input_csv_path,
    output_csv_path,
    base_audio_path
):
    modified_rows = []

    with open(input_csv_path, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)

        for row in reader:
            try:
                audio_path = os.path.join(base_audio_path, row['wav'].replace('.mp3', '.wav'))

                data, sampling_rate = sf.read(audio_path)
                duration = np.round(librosa.get_duration(y=data, sr=sampling_rate), 3)

                row['duration'] = duration
                row['wav'] = audio_path

                modified_rows.append(row)
            except Exception as e:
                print(f"Failed processing {row['wav']}: {e}")

        fieldnames = reader.fieldnames

    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(modified_rows)


In [ ]:
base_audio_path = '/content/accent-recog-slt2022/CommonAccent/data_small/wav_files/wav_files/'

# Train
update_csv_with_full_path_and_duration(
    input_csv_path='/content/accent-recog-slt2022/CommonAccent/data_small/train_orig.csv',
    output_csv_path='/content/accent-recog-slt2022/CommonAccent/data_small/train.csv',
    base_audio_path=base_audio_path
)

# Dev
update_csv_with_full_path_and_duration(
    input_csv_path='/content/accent-recog-slt2022/CommonAccent/data_small/dev_orig.csv',
    output_csv_path='/content/accent-recog-slt2022/CommonAccent/data_small/dev.csv',
    base_audio_path=base_audio_path
)

# Test
update_csv_with_full_path_and_duration(
    input_csv_path='/content/accent-recog-slt2022/CommonAccent/data_small/test_orig.csv',
    output_csv_path='/content/accent-recog-slt2022/CommonAccent/data_small/test.csv',
    base_audio_path=base_audio_path
)

In [ ]:
%cd /content/accent-recog-slt2022/CommonAccent
# make .sh file executable:
!chmod u+x run_accent_id_ecapa_tdnn.sh

/content/accent-recog-slt2022/CommonAccent


# Finally, we can run the model.
You can try to run the training using cpu by adding

--device="cpu"

to your .sh file (line 59)

In [ ]:
!./run_accent_id_ecapa_tdnn.sh

*** About to start the training ***
*** output folder: results/ECAPA-TDNN/EN/spkrec-ecapa-voxceleb/1986 ***
speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: results/ECAPA-TDNN/EN/spkrec-ecapa-voxceleb/1986
speechbrain.dataio.encoder - Load called, but CategoricalEncoder is not empty. Loaded data will overwrite everything. This is normal if there is e.g. an unk label defined at init.
speechbrain.pretrained.fetching - Fetch embedding_model.ckpt: Delegating to Huggingface hub, source speechbrain/spkrec-ecapa-voxceleb.
speechbrain.utils.parameter_transfer - Loading pretrained files for: embedding_model
speechbrain.core - Info: device arg overridden by command line input to: cuda:0
speechbrain.core - 20.8M trainable parameters in AID
speechbrain.utils.checkpoints - Would load a checkpoint here, but none found yet.
speechbrain.utils.epoch_loop - Going into epoch 1
100% 29/29 [00:32<00:00,  1.13s/it, train_loss=7.51]
100% 5/5 [00:02<00:00,  1.77it/s]
speechbrain.

In [ ]:
# set the correct pretrained_path: /content/accent-recog-slt2022/CommonAccent/results/ECAPA-TDNN/EN/spkrec-ecapa-voxceleb/1986/save/
# and load_folder: !ref <pretrained_path>/CKPT+2025-05-15+09-43-30+00
# in the yaml file!
# if run is successful, it will create an analysis folder (in your results folder)
!python accent_id/inference.py accent_id/hparams/inference_ecapa_tdnn.yaml

speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: results/analysis
speechbrain.dataio.encoder - Load called, but CategoricalEncoder is not empty. Loaded data will overwrite everything. This is normal if there is e.g. an unk label defined at init.
speechbrain.pretrained.fetching - Fetch embedding_model.ckpt: Using existing file/symlink in model_checkpoints/embedding_model.ckpt.
speechbrain.pretrained.fetching - Fetch classifier.ckpt: Using existing file/symlink in model_checkpoints/classifier.ckpt.
speechbrain.pretrained.fetching - Fetch accent_encoder.txt: Using existing file/symlink in model_checkpoints/label_encoder.ckpt.
speechbrain.utils.parameter_transfer - Loading pretrained files for: embedding_model, classifier, label_encoder
speechbrain.core - Info: device arg from hparam file is used
speechbrain.core - 20.8M trainable parameters in AccID_inf
100% 5/5 [00:02<00:00,  1.88it/s]
100% 5/5 [00:02<00:00,  2.47it/s]


# Our project starts here!

for 3 languages:
*   **Russian**
*   **Belarusian**
*   **Serbian**

# Serbian

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
import tarfile

tar_path = "/content/cv-corpus-21.0-2025-03-14-sr.tar" #your downloaded locally dataset
extract_path = "/content/cv_data"

with tarfile.open(tar_path, "r") as tar:
    tar.extractall(path=extract_path)

print("Extraction complete.")

Extraction complete.


In [23]:
!pip install -q speechbrain==0.5.13 \
    transformers==4.25.1 \
    librosa==0.9.2 \
    ipdb

In [24]:
import pandas as pd
import os

data_path = "/content/cv_data/cv-corpus-21.0-2025-03-14/sr"

validated = pd.read_csv(os.path.join(data_path, "validated.tsv"), sep='\t')
other = pd.read_csv(os.path.join(data_path, "other.tsv"), sep='\t')

print(f"Validated samples: {len(validated)}")
print(f"Other samples: {len(other)}")

df_full = pd.concat([validated, other], ignore_index=True)

print(f"Total samples combined: {len(df_full)}")

Validated samples: 6201
Other samples: 4252
Total samples combined: 10453


In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

data_path = "/content/cv_data/cv-corpus-21.0-2025-03-14/sr"
validated_path = os.path.join(data_path, "validated.tsv")
other_path = os.path.join(data_path, "other.tsv")

df_validated = pd.read_csv(validated_path, sep='\t')
df_other = pd.read_csv(other_path, sep='\t')

df_validated['source'] = 'validated'
df_other['source'] = 'other'

df_combined = pd.concat([df_validated, df_other], ignore_index=True)

print(f"Validated samples: {len(df_validated)}")
print(f"Other samples: {len(df_other)}")
print(f"Total samples combined: {len(df_combined)}")

# 70% train, 15% dev, 15% test
df_train_sr, df_temp_sr = train_test_split(df_combined, test_size=0.3, random_state=42)
df_dev_sr, df_test_sr = train_test_split(df_temp_sr, test_size=0.5, random_state=42)

print(f"Train size serbian: {len(df_train_sr)}")
print(f"Dev size serbian: {len(df_dev_sr)}")
print(f"Test size serbian: {len(df_test_sr)}")

Validated samples: 6201
Other samples: 4252
Total samples combined: 10453
Train size serbian: 7317
Dev size serbian: 1568
Test size serbian: 1568


In [26]:
df_train_sr.head()

,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment,source
3206,8871c9cc7b2845c3c9ada3fdcbe4570d946355312046c8...,common_voice_sr_36950568.mp3,5cbeabb5f1176eca6897f8b8418ecd6b23133789e36e99...,Он је мудра особа.,NaN,4,0,twenties,male_masculine,Južnjački,NaN,sr,NaN,validated
5616,c20cb0c0edb9cd9c40b412766f5b5d9a615df054ef83c7...,common_voice_sr_28441293.mp3,9eb009b7cf6218dd67f768b75dd75ac403eeb5896b1dde...,Зар ме не знате?,NaN,2,0,twenties,female_feminine,NaN,NaN,sr,NaN,validated
1860,4ad3d9184f039ff4d6070ed74e23707ae802e7a5ad545f...,common_voice_sr_39885228.mp3,01d85a691267486104607e391299696ac63e5cc3a1b6b7...,Жене га ипак воле!,NaN,4,0,twenties,NaN,šumajsko-vojvođanski,NaN,sr,NaN,validated
10210,6c24b4121862c2d34414d977ecf9134d1182f7bc686b05...,common_voice_sr_41024500.mp3,ef5510fc02e7234e9608168df23242657cd44f7f1c7b23...,Шта је ваш одговор?,NaN,0,0,sixties,NaN,NaN,NaN,sr,NaN,other
8131,6c24b4121862c2d34414d977ecf9134d1182f7bc686b05...,common_voice_sr_40981980.mp3,c2d823ccf7230fea77f1922518da8f1336c05c81a6f144...,Само молим вас?,NaN,0,0,sixties,NaN,NaN,NaN,sr,NaN,other


In [27]:
import os

input_wav_dir = '/content/accent-recog-slt2022/CommonAccent/data_small/wav_files'
all_wav_files = [f for f in os.listdir(input_wav_dir) if f.endswith('.wav')]
print(f"Всего wav в папке: {len(all_wav_files)}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/accent-recog-slt2022/CommonAccent/data_small/wav_files'

mp3 to wav

In [30]:
import os
import pandas as pd
import librosa
import soundfile as sf
from tqdm import tqdm

validated_path = '/content/cv_data/cv-corpus-21.0-2025-03-14/sr/validated.tsv'
other_path = '/content/cv_data/cv-corpus-21.0-2025-03-14/sr/other.tsv'

input_mp3_dir = '/content/cv_data/cv-corpus-21.0-2025-03-14/sr/clips'
output_wav_dir = '/content/accent-recog-slt2022/CommonAccent/data_small/wav_files'
os.makedirs(output_wav_dir, exist_ok=True)

df_validated = pd.read_csv(validated_path, sep='\t')
df_other = pd.read_csv(other_path, sep='\t')
df_all = pd.concat([df_validated, df_other], ignore_index=True)

all_paths = df_all['path'].unique()
print(f"Overall unique files for converting: {len(all_paths)}")

existing_files = [f for f in all_paths if os.path.exists(os.path.join(input_mp3_dir, f))]
print(f"Mp3 files exist on disk: {len(existing_files)}")

for file in tqdm(existing_files, desc="Converting MP3 -> WAV"):
    mp3_path = os.path.join(input_mp3_dir, file)
    wav_path = os.path.join(output_wav_dir, file.replace('.mp3', '.wav'))
    try:
        audio, sr = librosa.load(mp3_path, sr=16000, mono=True)
        sf.write(wav_path, audio, 16000)
    except Exception as e:
        print(f"❌ Error {file}: {e}")


Overall unique files for converting: 10453
Mp3 files exist on disk: 10453


Converting MP3 -> WAV: 100%|██████████| 10453/10453 [19:29<00:00,  8.94it/s]


In [32]:
import zipfile

# Step 1: Collect metadata (optional)
converted_files = []
for file in existing_files:
    wav_file = file.replace('.mp3', '.wav')
    wav_path = os.path.join(output_wav_dir, wav_file)
    if os.path.exists(wav_path):
        converted_files.append(wav_file)

# Save metadata CSV
df_converted = df_all[df_all['path'].isin(existing_files)].copy()
df_converted['path'] = df_converted['path'].apply(lambda x: x.replace('.mp3', '.wav'))
df_converted.to_csv('/content/converted_metadata_sr.csv', index=False)

# Step 2: Zip the output directory
zip_path = '/content/wav_dataset_sr.zip'
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for root, dirs, files in os.walk(output_wav_dir):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, output_wav_dir)
            zipf.write(file_path, arcname=arcname)

print("✅ Zipping complete.")

# Step 3: Create a download link (Google Colab-specific)
from google.colab import files
files.download(zip_path)


✅ Zipping complete.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [33]:
import pandas as pd
import os

validated_path = '/content/cv_data/cv-corpus-21.0-2025-03-14/sr/validated.tsv'
other_path = '/content/cv_data/cv-corpus-21.0-2025-03-14/sr/other.tsv'

df_validated = pd.read_csv(validated_path, sep='\t')
df_other = pd.read_csv(other_path, sep='\t')
df_all = pd.concat([df_validated, df_other], ignore_index=True)

keep_columns = ["path", "sentence", "client_id", "locale"]
df_all = df_all[[col for col in keep_columns if col in df_all.columns]]

wav_dir = '/content/accent-recog-slt2022/CommonAccent/data_small/wav_files'

def build_dataset(split_df, split_name):
    df_split = df_all[df_all["path"].isin(split_df["path"])].copy()

    df_split["path"] = df_split["path"].apply(lambda x: os.path.join(wav_dir, x.replace(".mp3", ".wav")))
    df_split = df_split[df_split["path"].apply(os.path.exists)]
    df_split["path"] = df_split["path"].apply(lambda x: os.path.relpath(x, start="/content"))

    df_split.rename(columns={
        "sentence": "text",
        "client_id": "speaker_id",
        "locale": "language"
    }, inplace=True)

    output_path = f"/content/{split_name}.sr.csv"
    df_split.to_csv(output_path, index=False)
    print(f"✅ {split_name}.ru.csv saved ({len(df_split)}).")

build_dataset(df_train_sr, "train")
build_dataset(df_dev_sr, "val")
build_dataset(df_test_sr, "test")

✅ train.ru.csv saved (7317).
✅ val.ru.csv saved (1568).
✅ test.ru.csv saved (1568).


In [36]:
pd.read_csv("/content/train.sr.csv").columns
import pandas as pd

df_train = pd.read_csv("/content/train.sr.csv")
df_train.head()

,path,text,speaker_id,language
0,accent-recog-slt2022/CommonAccent/data_small/w...,Само проширење међутим не представља окончање ...,b85fea5170493896b788360e6850087ed1712480443dde...,sr
1,accent-recog-slt2022/CommonAccent/data_small/w...,"Хенри се оженио и трећи пут, за краљицу изабра...",db9eeb26b34841ed37365a1e92f39b83b58bf8ca9b5242...,sr
2,accent-recog-slt2022/CommonAccent/data_small/w...,Реци и мени!,2d58fc9c2fa8c930ac0542e0b5e3812389045e509cf773...,sr
3,accent-recog-slt2022/CommonAccent/data_small/w...,"Хајде, хајде тамо.",2d58fc9c2fa8c930ac0542e0b5e3812389045e509cf773...,sr
4,accent-recog-slt2022/CommonAccent/data_small/w...,"Донели смо јакне, џемпере, топлију одећу.",4c7c196d5dc5168857ee8adfc2d1bb1f0a47211326ae26...,sr


# Russian

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
import tarfile

tar_path = "/content/cv-corpus-21.0-delta-2025-03-14-ru.tar" #your downloaded locally dataset
extract_path = "/content/cv_data"

with tarfile.open(tar_path, "r") as tar:
    tar.extractall(path=extract_path)

print("Extraction complete.")

Extraction complete.


In [86]:
!pip install -q speechbrain==0.5.13 \
    transformers==4.25.1 \
    librosa==0.9.2 \
    ipdb

In [87]:
import pandas as pd
import os

data_path = "/content/cv_data/cv-corpus-21.0-delta-2025-03-14/ru"

validated = pd.read_csv(os.path.join(data_path, "validated.tsv"), sep='\t')
other = pd.read_csv(os.path.join(data_path, "other.tsv"), sep='\t')

print(f"Validated samples: {len(validated)}")
print(f"Other samples: {len(other)}")

df_full = pd.concat([validated, other], ignore_index=True)

print(f"Total samples combined: {len(df_full)}")

Validated samples: 90
Other samples: 2940
Total samples combined: 3030


In [88]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

data_path = "/content/cv_data/cv-corpus-21.0-delta-2025-03-14/ru"
validated_path = os.path.join(data_path, "validated.tsv")
other_path = os.path.join(data_path, "other.tsv")

df_validated = pd.read_csv(validated_path, sep='\t')
df_other = pd.read_csv(other_path, sep='\t')

df_validated['source'] = 'validated'
df_other['source'] = 'other'

df_combined = pd.concat([df_validated, df_other], ignore_index=True)

print(f"Validated samples: {len(df_validated)}")
print(f"Other samples: {len(df_other)}")
print(f"Total samples combined: {len(df_combined)}")

# 70% train, 15% dev, 15% test
df_train_ru, df_temp_ru = train_test_split(df_combined, test_size=0.3, random_state=42)
df_dev_ru, df_test_ru = train_test_split(df_temp_ru, test_size=0.5, random_state=42)

print(f"Train size russian: {len(df_train_ru)}")
print(f"Dev size russian: {len(df_dev_ru)}")
print(f"Test size russian: {len(df_test_ru)}")

Validated samples: 90
Other samples: 2940
Total samples combined: 3030
Train size russian: 2121
Dev size russian: 454
Test size russian: 455


In [43]:
df_train_ru.head()

,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment,source
2546,d3f1f7c923f0522c03e865129fff2650df0b66ec0df81b...,common_voice_ru_42555795.mp3,dcfbc2f243aece89b99bf586adefc2238becf39f23ba60...,В разрабатываемой технологии взаимодействие ме...,NaN,0,0,NaN,NaN,NaN,NaN,ru,NaN,other
279,4a3d198ccac94aa3330809ce2eef552dec6625d1169508...,common_voice_ru_41973269.mp3,5939e5ac0fbe4b50bcf24d90f901e69ee50071e71be68f...,"Под летним небом звезды сверкали, словно алмаз...",NaN,0,0,thirties,NaN,"Georgian accent,Грузинский акцент",NaN,ru,NaN,other
1510,860c378a00fee36eb354361dae39730bfa969c53e99baf...,common_voice_ru_42327869.mp3,dc5309bb107302de9a63d31cfb8255131a5b42b0cc22ae...,Такой механизм применяется для урегулирования ...,NaN,0,0,thirties,male_masculine,NaN,NaN,ru,NaN,other
1553,860c378a00fee36eb354361dae39730bfa969c53e99baf...,common_voice_ru_42405536.mp3,dc7faf98f1e82b2f0e4ea8f8434fa4c63587a2e37045f0...,"Я смотрю, у вас новое назначение.",NaN,0,0,thirties,male_masculine,NaN,NaN,ru,NaN,other
2230,e4571c8ae7d79a96e6f28a3facacdd14a240bd3f246456...,common_voice_ru_42544239.mp3,d93b5579d81cc6b5087215517c8187a9a04b395cade88c...,Трансатлантическая работорговля была особенно ...,NaN,0,0,thirties,NaN,NaN,NaN,ru,NaN,other


In [90]:
import os

input_wav_dir = '/content/accent-recog-slt2022/CommonAccent/data_small/wav_files'
all_wav_files = [f for f in os.listdir(input_wav_dir) if f.endswith('.wav')]
print(f"Всего wav в папке: {len(all_wav_files)}")  # Ожидается ~3000



Всего wav в папке: 21325


Converting MP3 to WAV

In [91]:
import os
import pandas as pd
import librosa
import soundfile as sf
from tqdm import tqdm

validated_path = '/content/cv_data/cv-corpus-21.0-delta-2025-03-14/ru/validated.tsv'
other_path = '/content/cv_data/cv-corpus-21.0-delta-2025-03-14/ru/other.tsv'

input_mp3_dir = '/content/cv_data/cv-corpus-21.0-delta-2025-03-14/ru/clips'
output_wav_dir = '/content/accent-recog-slt2022/CommonAccent/data_small/wav_files'
os.makedirs(output_wav_dir, exist_ok=True)

df_validated = pd.read_csv(validated_path, sep='\t')
df_other = pd.read_csv(other_path, sep='\t')
df_all = pd.concat([df_validated, df_other], ignore_index=True)

all_paths = df_all['path'].unique()
print(f"Overall unique files for converting: {len(all_paths)}")

existing_files = [f for f in all_paths if os.path.exists(os.path.join(input_mp3_dir, f))]
print(f"Mp3 files exist on disk: {len(existing_files)}")

for file in tqdm(existing_files, desc="Converting MP3 -> WAV"):
    mp3_path = os.path.join(input_mp3_dir, file)
    wav_path = os.path.join(output_wav_dir, file.replace('.mp3', '.wav'))
    try:
        audio, sr = librosa.load(mp3_path, sr=16000, mono=True)
        sf.write(wav_path, audio, 16000)
    except Exception as e:
        print(f"❌ Error {file}: {e}")


Overall unique files for converting: 3030
Mp3 files exist on disk: 3030


Converting MP3 -> WAV: 100%|██████████| 3030/3030 [08:48<00:00,  5.74it/s]


In [ ]:
#import zipfile

#with zipfile.ZipFile("accent-recog-slt2022.zip", 'r') as zip_ref:
#    zip_ref.extractall("/content")


Creating train/test/val sets

In [92]:
import pandas as pd
import os

validated_path = '/content/cv_data/cv-corpus-21.0-delta-2025-03-14/ru/validated.tsv'
other_path = '/content/cv_data/cv-corpus-21.0-delta-2025-03-14/ru/other.tsv'

df_validated = pd.read_csv(validated_path, sep='\t')
df_other = pd.read_csv(other_path, sep='\t')
df_all = pd.concat([df_validated, df_other], ignore_index=True)

keep_columns = ["path", "sentence", "client_id", "locale"]
df_all = df_all[[col for col in keep_columns if col in df_all.columns]]

wav_dir = '/content/accent-recog-slt2022/CommonAccent/data_small/wav_files'

def build_dataset(split_df, split_name):
    df_split = df_all[df_all["path"].isin(split_df["path"])].copy()

    df_split["path"] = df_split["path"].apply(lambda x: os.path.join(wav_dir, x.replace(".mp3", ".wav")))
    df_split = df_split[df_split["path"].apply(os.path.exists)]
    df_split["path"] = df_split["path"].apply(lambda x: os.path.relpath(x, start="/content"))

    df_split.rename(columns={
        "sentence": "text",
        "client_id": "speaker_id",
        "locale": "language"
    }, inplace=True)

    output_path = f"/content/{split_name}.ru.csv"
    df_split.to_csv(output_path, index=False)
    print(f"✅ {split_name}.ru.csv saved ({len(df_split)}).")

build_dataset(df_train_ru, "train")
build_dataset(df_dev_ru, "val")
build_dataset(df_test_ru, "test")

✅ train.ru.csv saved (2121).
✅ val.ru.csv saved (454).
✅ test.ru.csv saved (455).


In [47]:
pd.read_csv("/content/train.ru.csv").columns


Index(['path', 'text', 'speaker_id', 'language'], dtype='object')

In [48]:
import pandas as pd

df_train = pd.read_csv("/content/train.ru.csv")
df_train.head()

,path,text,speaker_id,language
0,accent-recog-slt2022/CommonAccent/data_small/w...,Особенная кошка - любимица всей семьи,4a3d198ccac94aa3330809ce2eef552dec6625d1169508...,ru
1,accent-recog-slt2022/CommonAccent/data_small/w...,"Верный друг всегда поддержит в трудную минуту,...",4a3d198ccac94aa3330809ce2eef552dec6625d1169508...,ru
2,accent-recog-slt2022/CommonAccent/data_small/w...,"Помечтайте о мире, где каждый человек может пр...",4a3d198ccac94aa3330809ce2eef552dec6625d1169508...,ru
3,accent-recog-slt2022/CommonAccent/data_small/w...,Читать текст и пытаться звучать похоже на ориг...,4a3d198ccac94aa3330809ce2eef552dec6625d1169508...,ru
4,accent-recog-slt2022/CommonAccent/data_small/w...,"хотя все знают, что Венгрия подарила миру таки...",4a3d198ccac94aa3330809ce2eef552dec6625d1169508...,ru


# Belarusian

## loading the dataset locally

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #use this if you want to upload the file from colab

Mounted at /content/drive


In [93]:
import tarfile

tar_path = "/content/cv-corpus-21.0-delta-2025-03-14-be.tar" #your downloaded locally dataset
extract_path = "/content/cv_data"

with tarfile.open(tar_path, "r") as tar:
    tar.extractall(path=extract_path)

print("Extraction complete.")

Extraction complete.


In [ ]:
gdrive_path = "/content/drive/MyDrive/Colab Notebooks/cv-corpus-21.0-delta-2025-03-14-be.tar.gz" #change to your location on Google Drive - look it up in the directory from the left
#gdrive_path = "/content/drive/MyDrive/Colab Notebooks/cv-corpus-14.0-delta-2023-06-23-be.tar.gz" #a bigger file
extract_path = '/content/cv_data'
import tarfile

with tarfile.open(gdrive_path, "r:gz") as tar:
    tar.extractall(path=extract_path)

print("Extraction complete.")

Extraction complete.


In [ ]:
# importing from the directory
#import tarfile

#tar_path = "/content/cv-corpus-21.0-delta-2025-03-14-be.tar.gz" #your downloaded locally dataset
#extract_path = "/content/cv_data

#with tarfile.open(tar_path, "r:gz") as tar:
#    tar.extractall(path=extract_path)

#print("Extraction complete.")

In [51]:
!pip install -q speechbrain==0.5.13 \
    transformers==4.25.1 \
    librosa==0.9.2 \
    ipdb

In [94]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

data_path = "/content/cv_data/cv-corpus-21.0-delta-2025-03-14/be"
#data_path = "/content/cv_data/cv-corpus-14.0-delta-2023-06-23/be"
validated_path = os.path.join(data_path, "validated.tsv")
other_path = os.path.join(data_path, "other.tsv")

df_validated = pd.read_csv(validated_path, sep='\t')
df_other = pd.read_csv(other_path, sep='\t')

df_validated['source'] = 'validated'
df_other['source'] = 'other'

df_combined = pd.concat([df_validated, df_other], ignore_index=True)

print(f"Validated samples: {len(df_validated)}")
print(f"Other samples: {len(df_other)}")
print(f"Total samples combined: {len(df_combined)}\n")

# 70% train, 15% dev, 15% test
df_train_be, df_temp_be = train_test_split(df_combined, test_size=0.3, random_state=42)
df_dev_be, df_test_be = train_test_split(df_temp_be, test_size=0.5, random_state=42)

print(f"Train size belarusian: {len(df_train_be)}")
print(f"Dev size belarusian: {len(df_dev_be)}")
print(f"Test size belarusian: {len(df_test_be)}")

Validated samples: 639
Other samples: 7203
Total samples combined: 7842

Train size belarusian: 5489
Dev size belarusian: 1176
Test size belarusian: 1177


In [53]:
df_train_be.head()

,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment,source
605,94066a18a2f30a5f8c3b1b93908e9bd180797803cee8ce...,common_voice_be_42076166.mp3,b6b5d55f6e367536a6e794e0679d5ee421914d9fe8baa1...,"Да апошняга моманту цяжка было сказаць, хто ст...",NaN,2,0,NaN,NaN,NaN,NaN,be,NaN,validated
3196,659b04f837f6b33da64c1cfd5472c09c5c611dc8dec68f...,common_voice_be_42139499.mp3,b6ffebb4bb1af260ed3c3da87ffdc46b82da959c9e08d4...,"Дзе менавіта, пакуль няма ведама.",NaN,0,0,NaN,NaN,NaN,NaN,be,NaN,other
3779,eb1078ce1c52d7b2a283533cba8fed0042df12e131514f...,common_voice_be_42250413.mp3,b7802586d0187e30f20e7497341dd5ddc6e34cadf47ab9...,Аднак для навучальнага працэсу наяўнасць мален...,NaN,0,0,NaN,NaN,NaN,NaN,be,NaN,other
1468,98da89725e47cf9fd22a756f540a6d33817008d6624b86...,common_voice_be_41974615.mp3,b5fa7dc5a1765f5ea91af9365fe90116edd2e662ad0fed...,"Добра, што мы можам гандляваць ва ўсіх напрамках.",NaN,0,0,NaN,NaN,NaN,NaN,be,NaN,other
5784,659b04f837f6b33da64c1cfd5472c09c5c611dc8dec68f...,common_voice_be_42498438.mp3,b932fcdf2f0126a30008c8c5d9beedbf58e9dd3d8f3c05...,"Ён стаіць амаль у цэнтры райцэнтра, побач прах...",NaN,1,0,NaN,NaN,NaN,NaN,be,NaN,other


In [54]:
print(df_train_be["accents"])

605     NaN
3196    NaN
3779    NaN
1468    NaN
5784    NaN
       ... 
5226    NaN
5390    NaN
860     NaN
7603    NaN
7270    NaN
Name: accents, Length: 5489, dtype: object


In [55]:
be_accents_df = df_train_be["accents"].dropna()
print(be_accents_df)

4935        трасянка
416         трасянка
446         трасянка
4704        трасянка
4932        трасянка
            ...     
4933        трасянка
4929        трасянка
5088    Без акцэнту.
4931        трасянка
4659        трасянка
Name: accents, Length: 90, dtype: object


In [56]:
print(be_accents_df.value_counts())

accents
трасянка        87
Без акцэнту.     3
Name: count, dtype: int64


## processing audio

In my file I had mp3 files instead of wav files, so I have to convert them to be processable by speechbrain (may be I have not just found the information about its ability to process mp3 files)

In [95]:
import os
import librosa
import soundfile as sf
import pandas as pd
from tqdm import tqdm

input_mp3_dir = '/content/cv_data/cv-corpus-21.0-delta-2025-03-14/be/clips'
output_wav_dir = '/content/accent-recog-slt2022/CommonAccent/data_small/wav_files_2'
os.makedirs(output_wav_dir, exist_ok=True)

all_files = pd.concat([df_train_be, df_dev_be, df_test_be])["path"].unique()

# Filtering existing files
existing_files = [f for f in all_files if os.path.exists(os.path.join(input_mp3_dir, f))]
print(f"Files from CSV: {len(all_files)}, found: {len(existing_files)}")

# Convverting formats
for file in tqdm(existing_files, desc="Converting MP3 -> WAV"):
    mp3_path = os.path.join(input_mp3_dir, file)
    wav_path = os.path.join(output_wav_dir, file.replace('.mp3', '.wav'))

    try:
        audio, sr = librosa.load(mp3_path, sr=16000, mono=True)
        sf.write(wav_path, audio, 16000)
    except Exception as e:
        print(f"Could not process {file}: {e}")

Files from CSV: 7842, found: 7842


Converting MP3 -> WAV: 100%|██████████| 7842/7842 [19:30<00:00,  6.70it/s]


remaining all columns

In [58]:
wav_folder = output_wav_dir  # path to saved .wav files
for df in [df_train_be, df_dev_be, df_test_be]:
    df["wav"] = df["path"].apply(lambda x: os.path.join(wav_folder, x.replace(".mp3", ".wav")))

In [59]:
df_train_be.head()

,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment,source,wav
605,94066a18a2f30a5f8c3b1b93908e9bd180797803cee8ce...,common_voice_be_42076166.mp3,b6b5d55f6e367536a6e794e0679d5ee421914d9fe8baa1...,"Да апошняга моманту цяжка было сказаць, хто ст...",NaN,2,0,NaN,NaN,NaN,NaN,be,NaN,validated,/content/accent-recog-slt2022/CommonAccent/dat...
3196,659b04f837f6b33da64c1cfd5472c09c5c611dc8dec68f...,common_voice_be_42139499.mp3,b6ffebb4bb1af260ed3c3da87ffdc46b82da959c9e08d4...,"Дзе менавіта, пакуль няма ведама.",NaN,0,0,NaN,NaN,NaN,NaN,be,NaN,other,/content/accent-recog-slt2022/CommonAccent/dat...
3779,eb1078ce1c52d7b2a283533cba8fed0042df12e131514f...,common_voice_be_42250413.mp3,b7802586d0187e30f20e7497341dd5ddc6e34cadf47ab9...,Аднак для навучальнага працэсу наяўнасць мален...,NaN,0,0,NaN,NaN,NaN,NaN,be,NaN,other,/content/accent-recog-slt2022/CommonAccent/dat...
1468,98da89725e47cf9fd22a756f540a6d33817008d6624b86...,common_voice_be_41974615.mp3,b5fa7dc5a1765f5ea91af9365fe90116edd2e662ad0fed...,"Добра, што мы можам гандляваць ва ўсіх напрамках.",NaN,0,0,NaN,NaN,NaN,NaN,be,NaN,other,/content/accent-recog-slt2022/CommonAccent/dat...
5784,659b04f837f6b33da64c1cfd5472c09c5c611dc8dec68f...,common_voice_be_42498438.mp3,b932fcdf2f0126a30008c8c5d9beedbf58e9dd3d8f3c05...,"Ён стаіць амаль у цэнтры райцэнтра, побач прах...",NaN,1,0,NaN,NaN,NaN,NaN,be,NaN,other,/content/accent-recog-slt2022/CommonAccent/dat...


In [96]:
# Save to the path your later code expects
output_dir = "/content/data_all_columns"
os.makedirs(output_dir, exist_ok=True)
df_train_be.to_csv(os.path.join(output_dir, "train.be.csv"), index=False)
df_dev_be.to_csv(os.path.join(output_dir, "dev.be.csv"), index=False)
df_test_be.to_csv(os.path.join(output_dir, "test.be.csv"), index=False)

with removing some columns

In [97]:
import pandas as pd
import os

validated_path = '/content/cv_data/cv-corpus-21.0-delta-2025-03-14/be/validated.tsv'
other_path = '/content/cv_data/cv-corpus-21.0-delta-2025-03-14/be/other.tsv'

df_validated = pd.read_csv(validated_path, sep='\t')
df_other = pd.read_csv(other_path, sep='\t')
df_all = pd.concat([df_validated, df_other], ignore_index=True)

keep_columns = ["path", "sentence", "client_id", "locale"]
df_all = df_all[[col for col in keep_columns if col in df_all.columns]]

output_wav_dir = '/content/accent-recog-slt2022/CommonAccent/data_small/wav_files'

def build_dataset(split_df, split_name):
    df_split = df_all[df_all["path"].isin(split_df["path"])].copy()

    df_split["path"] = df_split["path"].apply(lambda x: os.path.join(output_wav_dir, x.replace(".mp3", ".wav")))
    df_split = df_split[df_split["path"].apply(os.path.exists)]
    df_split["path"] = df_split["path"].apply(lambda x: os.path.relpath(x, start="/content"))

    df_split.rename(columns={
        "sentence": "text",
        "client_id": "speaker_id",
        "locale": "language"
    }, inplace=True)

    output_path = f"/content/{split_name}.be2.csv"
    df_split.to_csv(output_path, index=False)
    print(f"{split_name}.be2.csv saved ({len(df_split)}).")

build_dataset(df_train_be, "train")
build_dataset(df_dev_be, "val")
build_dataset(df_test_be, "test")

train.be2.csv saved (5489).
val.be2.csv saved (1176).
test.be2.csv saved (1177).


In [98]:
import pandas as pd

# Load the datasets
df_train = pd.read_csv("/content/train.be2.csv")
df_val = pd.read_csv("/content/val.be2.csv")
df_test = pd.read_csv("/content/test.be2.csv")

print("Train:")
df_train.head()

Train:


,path,text,speaker_id,language
0,accent-recog-slt2022/CommonAccent/data_small/w...,"Прасцей кажучы, пустыя паліцы, рост коштаў і г...",1fae492f6bb5d39df233deca5189c444033e2653aecc51...,be
1,accent-recog-slt2022/CommonAccent/data_small/w...,Бабулі ж і ўпрочкі бегаць не было куды.,4395656b63879aa58eff64fe74ccb6a892a4af39c274dd...,be
2,accent-recog-slt2022/CommonAccent/data_small/w...,На што вам ведаць пра што гэты спектакль?,964253dda7b98805d78fcc2a24cef3c53b5e2b53a4c89f...,be
3,accent-recog-slt2022/CommonAccent/data_small/w...,Панскіх яблыкаў табе шкода?,9e2592f21f4f711c38affd76643d43a64128fa51609be4...,be
4,accent-recog-slt2022/CommonAccent/data_small/w...,Пры некіравальным спуску судна перамяшчаецца п...,a133c0323e3032e2b1aede8a367c31f59cdc6601abfaf6...,be


In [ ]:
!zip -r /content/data_small.zip /content/accent-recog-slt2022/CommonAccent/data_small

from google.colab import files
files.download("/content/data_small.zip")

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
updating: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/common_voice_sr_40977474.wav (deflated 50%)
updating: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/common_voice_sr_40979017.wav (deflated 31%)
updating: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/common_voice_sr_40991296.wav (deflated 25%)
updating: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/common_voice_sr_38535706.wav (deflated 24%)
updating: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/common_voice_be_42260872.wav (deflated 40%)
updating: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/common_voice_sr_35089136.wav (deflated 63%)
updating: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/common_voice_be_41929969.wav (deflated 49%)
updating: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/common_voice_be_42313881.wav (deflated 21

In [64]:
!cp -R /content/accent-recog-slt2022/CommonAccent/data_small/wav_files /content/drive/MyDrive/.../data/wav

cp: cannot create directory '/content/drive/MyDrive/.../data/wav': No such file or directory


## Accent extraction - failure

common_accent_prepare.py is a script in the loaded repo
it is situated here: /content/accent-recog-slt2022/CommonAccent/common_accent_prepare.py

```
def prepare_common_accent(
        data_folder,
        save_folder,
        accented_letters=False,
        language="en",        
        skip_prep=False,
    ):

    """
    Prepares the csv files for the CommonAccent dataset for Accent Classification.
    Download: https://commonvoice.mozilla.org/en/datasets

    Arguments
    ---------
    data_folder : str
        Path to the folder where the CommonAccent dataset for Accent Classification is stored.
        This path should include the multi: /datasets/CommonAccent
    save_folder : str
        The directory where to store the csv files.
    max_duration : int, optional
        Max duration (in seconds) of training uterances.
    accented_letters : bool, optional
        Defines if accented letters will be kept as individual letters or
        transformed to the closest non-accented letters.
    language: str
        Specify the language for text normalization.        
    skip_prep: bool
        If True, skip data preparation.

    Example
    -------
    >>> from recipes.CommonAccent.common_accent_prepare import prepare_common_accent
    >>> data_folder = '/datasets/CommonAccent'
    >>> save_folder = 'exp/CommonAccent_exp'
    >>> prepare_common_accent(\
            data_folder,\
            save_folder,\
            skip_prep=False\
        )
    """
```


In [65]:
import sys
sys.path.append("/content/accent-recog-slt2022")

from CommonAccent.common_accent_prepare import prepare_common_accent

prepare_common_accent(
    data_folder="/content/cv_data/",
    save_folder="/content/common_accent_output",
    language="be",
    skip_prep=False
)

ModuleNotFoundError: No module named 'CommonAccent.common_accent_prepare'

In [66]:
!python common_accent_prepare.py --language "be" /content/cv_data /content/common_accent_output

python3: can't open file '/content/common_accent_prepare.py': [Errno 2] No such file or directory


Unfortunatlelly, the scripts doen ot have an option for our chosen langauge, so we decided to focus on just langauge recognition of Russian, Belarusian and Serbian

## Optional: loading from HuggingFace

download_data_hf.py is a script in the loaded repo
it is situated here: /content/accent-recog-slt2022/CommonAccent/download_data_hf.py

**use the code from HugginFace and the number of the dataset you need, e.g.19  as in the script**
https://huggingface.co/datasets/fsicoli/common_voice_19_0


```
"""
Script to download the CommonVoice dataset using Hugging Face

At Hugging Face: https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0
Download the dataset: https://commonvoice.mozilla.org/en/datasets

def prepare_cv_from_hf(output_folder, language="en"):
    """ function to prepare the datasets in <output-folder> """

    output_folder = os.path.join(output_folder, language)
    # create the output folder: in case is not present
    os.makedirs(output_folder, exist_ok=True)

    # Prepare the the common voice dataset in streaming mode
    # common_voice_ds = load_dataset(_COMMON_VOICE_FOLDER, language, streaming=True)
    common_voice_ds = load_dataset("fsicoli/common_voice_19_0", language, streaming=True, trust_remote_code=True)

    !!!!
    it uses here already a dataset

    # just select relevant splits: train/validation/test set
    splits = ["train", "validation", "test"]
    common_voice = {}
    
    # load, prepare and filter each split in streaming mode:
    for split in splits:
        # filter out samples without accent
        ds = common_voice_ds[split].filter( lambda x: x['accent'] != '')
        common_voice[split] = ds
        
    for dataset in common_voice:
        csv_lines = []
        # Starting index
        idx = 0
        for sample in common_voice[dataset]:
            # get path and utt_id
            mp3_path = sample['path']
            utt_id = mp3_path.split(".")[-2].split("/")[-1]            
            
            # Create a row with metadata + transcripts
            csv_line = [
                idx,  # ID
                utt_id,  # Utterance ID
                mp3_path,  # File name
                sample["locale"],
                sample["accent"],
                sample["age"],
                sample["gender"],
                sample["sentence"], # transcript
            ]

            # Adding this line to the csv_lines list
            csv_lines.append(csv_line)
            # Increment index
            idx += 1

        # CSV column titles
        csv_header = ["idx", "utt_id", "mp3_path", "language", "accent", "age", "gender", "transcript"]
        # Add titles to the list at indexx 0
        csv_lines.insert(0, csv_header)

        # Writing the csv lines
        csv_file = os.path.join(output_folder, dataset+'.tsv')

        with open(csv_file, mode="w", encoding="utf-8") as csv_f:
            csv_writer = csv.writer(
                csv_f, delimiter="\t", quotechar='"', quoting=csv.QUOTE_MINIMAL
            )
            for line in csv_lines:
                csv_writer.writerow(line)
    print(f"Prepare CommonVoice: for {language} in {output_folder}")

def main():
    # read input from CLI, you need to run it from the command lind
    parser = argparse.ArgumentParser()

    # reporting vars
    parser.add_argument(
        "--language",
        type=str,
        default='en',
        help="Language to load",
    )
    parser.add_argument(
        "output_folder",
        help="path of the output folder to store the csv files for each split",
    )
    args = parser.parse_args()
```



**So, the script loads already one dataset
    common_voice_ds = load_dataset("fsicoli/common_voice_19_0", language, streaming=True, trust_remote_code=True), therefore you can run the cell below the same as it was in the original notebook.
    If you need, however, abother dataset, e.g. wit hmore audio files, you need copy the code above and specify it and run the code without a script but your code in the cells**

In [ ]:
#from datasets import load_dataset
#common_voice_ds = load_dataset("mozilla-foundation/common_voice_19_0", "be")

In [ ]:
def prepare_cv_from_hf(output_folder, language="en"):

    output_folder = os.path.join(output_folder, language)
    # create the output folder: in case is not present
    os.makedirs(output_folder, exist_ok=True)

    # Prepare the the common voice dataset in streaming mode
    common_voice_ds = load_dataset(_COMMON_VOICE_FOLDER, language, streaming=True)
    #common_voice_ds = load_dataset("fsicoli/common_voice_19_0", language, streaming=True, trust_remote_code=True)

    # just select relevant splits: train/validation/test set
    splits = ["train", "validation", "test"]
    common_voice = {}

    # load, prepare and filter each split in streaming mode:
    for split in splits:
        # filter out samples without accent
        ds = common_voice_ds[split].filter( lambda x: x['accent'] != '')
        common_voice[split] = ds

    for dataset in common_voice:
        csv_lines = []
        # Starting index
        idx = 0
        for sample in common_voice[dataset]:
            # get path and utt_id
            mp3_path = sample['path']
            utt_id = mp3_path.split(".")[-2].split("/")[-1]

            # Create a row with metadata + transcripts
            csv_line = [
                idx,  # ID
                utt_id,  # Utterance ID
                mp3_path,  # File name
                sample["locale"],
                sample["accent"],
                sample["age"],
                sample["gender"],
                sample["sentence"], # transcript
            ]

            # Adding this line to the csv_lines list
            csv_lines.append(csv_line)
            # Increment index
            idx += 1

        # CSV column titles
        csv_header = ["idx", "utt_id", "mp3_path", "language", "accent", "age", "gender", "transcript"]
        # Add titles to the list at indexx 0
        csv_lines.insert(0, csv_header)

        # Writing the csv lines
        csv_file = os.path.join(output_folder, dataset+'.tsv')

        with open(csv_file, mode="w", encoding="utf-8") as csv_f:
            csv_writer = csv.writer(
                csv_f, delimiter="\t", quotechar='"', quoting=csv.QUOTE_MINIMAL
            )
            for line in csv_lines:
                csv_writer.writerow(line)
    print(f"Prepare CommonVoice: for {language} in {output_folder}")

In [ ]:
#prepare_cv_from_hf("/content/cv_data", language="be", data_dir="/content/cv_data/cv-corpus-14.0-delta-2023-06-23/be") still loads the HugginFace

In [ ]:
#!python /content/accent-recog-slt2022/CommonAccent/download_data_hf.py --language "be" /content/cv_data #loads friscoli/common_voice 19

#it is an original code

python3: can't open file '/content/accent-recog-slt2022/CommonAccent/download_data_hf.py': [Errno 2] No such file or directory


In [ ]:
import os
import pandas as pd
import torchaudio
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from scipy.io.wavfile import write
from datasets import load_dataset

# Paths
data_path = "/content/accent-recog-slt2022/CommonAccent/data/"
audio_root = "/content/cv_data/cv-corpus-21.0-delta-2025-03-14/be/clips/"
save_dir = os.path.join(data_path, "wav_files")
os.makedirs(save_dir, exist_ok=True)

# Load CSVs
df_train = pd.read_csv(os.path.join(data_path, "train.csv"))
df_test = pd.read_csv(os.path.join(data_path, "test.csv"))
df_dev = pd.read_csv(os.path.join(data_path, "dev.csv"))

# Combine all file references
all_files = pd.concat([df_train, df_test, df_dev])
unique_filenames = all_files["wav"].apply(os.path.basename).unique()

# Convert and save as .wav
for file_name in tqdm(unique_filenames, desc="Converting audio"):
    mp3_path = os.path.join(audio_root, file_name)
    if not os.path.isfile(mp3_path):
        print(f"Missing file: {mp3_path}")
        continue
    try:
        waveform, sample_rate = torchaudio.load(mp3_path)
        wav_path = os.path.join(save_dir, file_name.replace(".mp3", ".wav"))
        torchaudio.save(wav_path, waveform, sample_rate)
    except Exception as e:
        print(f"Error processing {mp3_path}: {e}")

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.io.wavfile import write
from datasets import load_dataset

data_path = "/content/accent-recog-slt2022/CommonAccent/data/"

# Load CSV
csv_path_train = data_path + "train.csv"
df_train = pd.read_csv(csv_path_train)
file_list_train = list(set(df_train['wav']))

csv_path_test = data_path + "test.csv"
df_test = pd.read_csv(csv_path_test)
file_list_test = list(set(df_test['wav']))

csv_path_dev = data_path + "dev.csv"
df_dev = pd.read_csv(csv_path_dev)
file_list_dev = list(set(df_dev['wav']))

# Set save directory
save_dir = data_path + "wav_files"
os.makedirs(save_dir, exist_ok=True)

# Load dataset in streaming mode
language = "be"  # or whatever language code you're using
cv_11_be_train = load_dataset("mozilla-foundation/common_voice_11_0", language, streaming=True, split="train", trust_remote_code=True)
print(next(iter(cv_11_be_train)))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Reading metadata...: 171388it [00:07, 23662.65it/s]


{'client_id': 'c14f21cacf2b7701ad0bead0dd1b31ec9d3a1557708e446de55e98b4b470cf31072c82543e5ba518c5c187a91868878a4e32727054a3dd94f9df41c9a13d8c62', 'path': 'it_train_0/common_voice_it_17415777.mp3', 'audio': {'path': 'it_train_0/common_voice_it_17415777.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -1.61342832e-05, -1.57291943e-05, -1.16234423e-05]), 'sampling_rate': 48000}, 'sentence': "Il marchese aveva già moglie in quell'epoca?", 'up_votes': 3, 'down_votes': 0, 'age': '', 'gender': '', 'accent': '', 'locale': 'it', 'segment': '', 'variant': ''}


In [ ]:
cv_11_be_dev = load_dataset("mozilla-foundation/common_voice_11_0", language, streaming=False, split="validation", trust_remote_code=True)
cv_11_be_test = load_dataset("mozilla-foundation/common_voice_11_0", language, streaming=False, split="test", trust_remote_code=True)

In [ ]:
import pandas as pd
# Count entries per accent
accent_counts = df_train['accent'].value_counts()

# Print the result
print(accent_counts)

# select randomly max 300 samples for each accent that has more than 300 samples

accent
TENDENTE AL SICULO MA NON MARCATO    1801
BASILICATA TRENTINO                  1801
VENETO                               1393
MERIDIONALE                           134
EMILIANO                               91
Name: count, dtype: int64


## Creating a common dataset (be + ru + ser)

I have deleted code for balancing because we don't need it. Instead we need to combine our train/test/val sets, so that we coul train the model in the next step.

My russian CSVs contain the following: path, text, speaker_id, language


In [73]:
import os
import pandas as pd

base_path = "/content"

df_train_ru = pd.read_csv("/content/train.ru.csv")
df_val_ru = pd.read_csv("/content/val.ru.csv")
df_test_ru = pd.read_csv("/content/test.ru.csv")

df_train_be = pd.read_csv(os.path.join(base_path, "train.be2.csv"))
df_val_be = pd.read_csv(os.path.join(base_path, "val.be2.csv"))
df_test_be = pd.read_csv(os.path.join(base_path, "test.be2.csv"))

df_train_sr = pd.read_csv(os.path.join(base_path, "train.sr.csv"))
df_val_sr = pd.read_csv(os.path.join(base_path, "val.sr.csv"))
df_test_sr = pd.read_csv(os.path.join(base_path, "test.sr.csv"))


In [74]:
df_train_all = pd.concat([df_train_ru, df_train_be, df_train_sr], ignore_index=True)
df_val_all = pd.concat([df_val_ru, df_val_be, df_val_sr], ignore_index=True)
df_test_all = pd.concat([df_test_ru, df_test_be, df_test_sr], ignore_index=True)

#df_train_all = pd.concat([df_train_ru, df_train_be], ignore_index=True)
#df_val_all = pd.concat([df_val_ru, df_val_be], ignore_index=True)
#df_test_all = pd.concat([df_test_ru, df_test_be], ignore_index=True)


In [75]:
output_path = "/content/data_all"

os.makedirs(output_path, exist_ok=True)

df_train_all.to_csv(os.path.join(output_path, "train_all.csv"), index=False)
df_val_all.to_csv(os.path.join(output_path, "val_all.csv"), index=False)
df_test_all.to_csv(os.path.join(output_path, "test_all.csv"), index=False)

print("✅ All CSVs are combined and saved")


✅ All CSVs are combined and saved


In [76]:
lang = df_train_all["language"].dropna()

lang_counts = lang.value_counts()

print(lang_counts)

language
sr    7317
be    5489
ru    2121
Name: count, dtype: int64


In [77]:
df_train_all.head()

,path,text,speaker_id,language
0,accent-recog-slt2022/CommonAccent/data_small/w...,Особенная кошка - любимица всей семьи,4a3d198ccac94aa3330809ce2eef552dec6625d1169508...,ru
1,accent-recog-slt2022/CommonAccent/data_small/w...,"Верный друг всегда поддержит в трудную минуту,...",4a3d198ccac94aa3330809ce2eef552dec6625d1169508...,ru
2,accent-recog-slt2022/CommonAccent/data_small/w...,"Помечтайте о мире, где каждый человек может пр...",4a3d198ccac94aa3330809ce2eef552dec6625d1169508...,ru
3,accent-recog-slt2022/CommonAccent/data_small/w...,Читать текст и пытаться звучать похоже на ориг...,4a3d198ccac94aa3330809ce2eef552dec6625d1169508...,ru
4,accent-recog-slt2022/CommonAccent/data_small/w...,"хотя все знают, что Венгрия подарила миру таки...",4a3d198ccac94aa3330809ce2eef552dec6625d1169508...,ru


In [84]:
import zipfile
import os

zip_path = "/content/data_small.zip"
temp_extract_path = "/content/temp_data_small_be"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(temp_extract_path)


zip_path = "/content/accent-recog-slt2022/CommonAccent/data_small/wav_files.zip"
temp_extract_path = "/content/temp_data_small_ru"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(temp_extract_path)

zip_path = "/content/wav_dataset_sr.zip"
temp_extract_path = "/content/temp_data_small_sr"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(temp_extract_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/accent-recog-slt2022/CommonAccent/data_small/wav_files.zip'

In [ ]:
import os
import shutil

ru_wav_dir = "/content/temp_data_small_ru/content/accent-recog-slt2022/CommonAccent/data_small/wav_files"
be_wav_dir = "/content/temp_data_small_be/content/accent-recog-slt2022/CommonAccent/data_small/wav_files"

combined_wav_dir = "/content/data_all/wav_files"
os.makedirs(combined_wav_dir, exist_ok=True)

def copy_wavs(src_dir, dst_dir):
    copied = 0
    for fname in os.listdir(src_dir):
        src_path = os.path.join(src_dir, fname)
        dst_path = os.path.join(dst_dir, fname)
        if not os.path.exists(dst_path):
            shutil.copy2(src_path, dst_path)
            copied += 1
    return copied

ru_copied = copy_wavs(ru_wav_dir, combined_wav_dir)
be_copied = copy_wavs(be_wav_dir, combined_wav_dir)

print(f"✅ Copied {ru_copied} russian and {be_copied} belorusian audiofiles in {combined_wav_dir}")



✅ Copied 3030 russian and 7843 belorusian audiofiles in /content/data_all/wav_files


In [ ]:
!zip -r /content/wav_files.zip /content/accent-recog-slt2022/CommonAccent/data_small/wav_files


  adding: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/ (stored 0%)
  adding: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/common_voice_ru_41973978.wav (deflated 28%)
  adding: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/common_voice_ru_41915177.wav (deflated 37%)
  adding: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/common_voice_ru_41973194.wav (deflated 49%)
  adding: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/common_voice_ru_42032048.wav (deflated 35%)
  adding: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/common_voice_ru_42544309.wav (deflated 35%)
  adding: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/common_voice_ru_42473293.wav (deflated 27%)
  adding: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/common_voice_ru_42619066.wav (deflated 29%)
  adding: content/accent-recog-slt2022/CommonAccent/data_small/wav_files/common_voice_ru_424154

In [ ]:
from google.colab import files
files.download('/content/wav_files.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## training

In [ ]:
!pip install -q pandas==2.2.2 \
    huggingface_hub>=0.7.0 \
    hyperpyyaml==0.0.1 \
    joblib \
    numpy==2.0.2 \
    packaging \
    pre-commit==2.3.0 \
    sentencepiece>=0.1.91 \
    SoundFile \
    tqdm

!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 848.7/848.7 MB 677.0 kB/s eta 0:00:00


In [ ]:
def update_csv_with_full_path_and_duration(input_csv_path, output_csv_path, base_audio_path):
    import csv
    import os
    import soundfile as sf
    import librosa
    import numpy as np

    modified_rows = []

    with open(input_csv_path, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames

        wav_key = None
        for key in reader.fieldnames:
            if 'wav' in key.lower() or 'path' in key.lower() or 'file' in key.lower():
                wav_key = key
                break

        if wav_key is None:
            raise ValueError("❌ Audio name not found. Make sure there is a column 'wav' or 'path'.")

        for row in reader:
            try:
                filename = os.path.basename(row[wav_key]).replace('.mp3', '.wav')
                audio_path = os.path.join(base_audio_path, filename)
                data, sr = sf.read(audio_path)
                duration = np.round(librosa.get_duration(y=data, sr=sr), 3)
                row['duration'] = duration
                row[wav_key] = audio_path
                modified_rows.append(row)
            except Exception as e:
                print(f"❌ Failed: {row.get(wav_key, 'N/A')}: {e}")

    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=fieldnames + ['duration'])
        writer.writeheader()
        writer.writerows(modified_rows)




In [ ]:
import csv
import os
import soundfile as sf

base_audio_path = '/content/data_all/wav_files'

update_csv_with_full_path_and_duration('/content/data_all/train_all.csv', '/content/train_updated.csv', base_audio_path)
update_csv_with_full_path_and_duration('/content/data_all/val_all.csv',   '/content/dev_updated.csv',   base_audio_path)
update_csv_with_full_path_and_duration('/content/data_all/test_all.csv',  '/content/test_updated.csv',  base_audio_path)


In [ ]:
pd.read_csv('/content/train_updated.csv').head()


,path,text,speaker_id,language,duration
0,/content/data_all/wav_files/common_voice_ru_41...,Особенная кошка - любимица всей семьи,4a3d198ccac94aa3330809ce2eef552dec6625d1169508...,ru,3.852
1,/content/data_all/wav_files/common_voice_ru_41...,"Верный друг всегда поддержит в трудную минуту,...",4a3d198ccac94aa3330809ce2eef552dec6625d1169508...,ru,5.688
2,/content/data_all/wav_files/common_voice_ru_41...,"Помечтайте о мире, где каждый человек может пр...",4a3d198ccac94aa3330809ce2eef552dec6625d1169508...,ru,4.536
3,/content/data_all/wav_files/common_voice_ru_41...,Читать текст и пытаться звучать похоже на ориг...,4a3d198ccac94aa3330809ce2eef552dec6625d1169508...,ru,5.328
4,/content/data_all/wav_files/common_voice_ru_41...,"хотя все знают, что Венгрия подарила миру таки...",4a3d198ccac94aa3330809ce2eef552dec6625d1169508...,ru,5.904


In [ ]:
!pip install -q speechbrain pandas librosa soundfile


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 22.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd

for fname in ['train_updated.csv', 'dev_updated.csv', 'test_updated.csv']:
    df = pd.read_csv(f'/content/{fname}')
    df['ID'] = df.index.astype(str)  # create ID
    df.to_csv(f'/content/{fname}', index=False)

In [ ]:
import pandas as pd
import os

# Loading data
train_df = pd.read_csv('/content/train_updated.csv')
valid_df = pd.read_csv('/content/dev_updated.csv')
test_df  = pd.read_csv('/content/test_updated.csv')

# Check the structure
print(train_df.columns)
print(train_df.head())


Index(['path', 'text', 'speaker_id', 'language', 'duration', 'ID'], dtype='object')
                                                path  \
0  /content/data_all/wav_files/common_voice_ru_41...   
1  /content/data_all/wav_files/common_voice_ru_41...   
2  /content/data_all/wav_files/common_voice_ru_41...   
3  /content/data_all/wav_files/common_voice_ru_41...   
4  /content/data_all/wav_files/common_voice_ru_41...   

                                                text  \
0              Особенная кошка - любимица всей семьи   
1  Верный друг всегда поддержит в трудную минуту,...   
2  Помечтайте о мире, где каждый человек может пр...   
3  Читать текст и пытаться звучать похоже на ориг...   
4  хотя все знают, что Венгрия подарила миру таки...   

                                          speaker_id language  duration  ID  
0  4a3d198ccac94aa3330809ce2eef552dec6625d1169508...       ru     3.852   0  
1  4a3d198ccac94aa3330809ce2eef552dec6625d1169508...       ru     5.688   1  
2  4a3d1

In [ ]:
!pip install -q speechbrain pandas librosa soundfile torchaudio


In [ ]:
import sys
print([p for p in sys.path])
import os
print(os.listdir('.'))


['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython']
['.config', 'train.ru.csv', '=0.7.0', 'test.be2.csv', 'data_all', 'test_updated.csv', 'val.be2.csv', 'temp_data_small_ru', 'val.ru.csv', 'train.be2.csv', 'test.ru.csv', 'data_small.zip', 'dev_updated.csv', 'train_updated.csv', 'wav_files.zip', 'temp_data_small_be', '=0.1.91', 'sample_data']


In [ ]:
#import importlib.util
#spec = importlib.util.find_spec("torchaudio")
#print(spec.origin)


/usr/local/lib/python3.11/dist-packages/torchaudio/__init__.py


In [ ]:
#pip uninstall torchaudio -y


Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124


In [ ]:
#pip install torch torchaudio --upgrade --force-reinstall


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 799.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [ ]:
#import torchaudio
#print(torchaudio.__version__)
#print(torchaudio.lib._torchaudio)


2.7.1+cu126
<module 'torchaudio.lib._torchaudio' from '/usr/local/lib/python3.11/dist-packages/torchaudio/lib/_torchaudio.so'>


In [ ]:
import torch
import torchaudio
import pandas as pd
import speechbrain as sb
from speechbrain import Brain
from speechbrain.dataio.dataset import DynamicItemDataset
from speechbrain.dataio.encoder import CategoricalEncoder
from speechbrain.lobes.models.Xvector import Xvector
from speechbrain.nnet.linear import Linear
from speechbrain.processing.features import STFT, Filterbank, InputNormalization

CSV_TRAIN = "/content/train_updated.csv"
CSV_VALID = "/content/dev_updated.csv"
CSV_TEST  = "/content/test_updated.csv"

sample_rate = 16000
n_mels = 40
num_languages = 3  # be, ru, sr

lang_encoder = CategoricalEncoder()
lang_encoder.update_from_iterable(["be", "ru", "sr"])

def audio_pipeline(path, max_len=48000):
    sig, fs = torchaudio.load(path)
    sig = sig.squeeze(0)
    if sig.shape[0] > max_len:
        sig = sig[:max_len]
    else:
        sig = torch.nn.functional.pad(sig, (0, max_len - sig.shape[0]))
    return sig

def prepare_dataset(csv_path):
    dataset = DynamicItemDataset.from_csv(csv_path)
    dataset.add_dynamic_item(audio_pipeline, takes=["path"], provides=["sig"])

    def encode_language(language):
        return lang_encoder.encode_label(language)

    dataset.add_dynamic_item(encode_language, takes=["language"], provides=["lang_encoded"])
    dataset.set_output_keys(["sig", "lang_encoded"])
    return dataset

train_data = prepare_dataset(CSV_TRAIN)
valid_data = prepare_dataset(CSV_VALID)
test_data  = prepare_dataset(CSV_TEST)

compute_STFT = STFT(sample_rate=sample_rate)
compute_fbanks = Filterbank(n_mels=n_mels)
normalize = InputNormalization()

embedding_model = Xvector()

import torch
import torch.nn as nn

class StatsPooling(nn.Module):
    def forward(self, x):
        mean = x.mean(dim=2)
        std = x.std(dim=2)
        return torch.cat((mean, std), dim=1)

class FixedXvector(nn.Module):
    def __init__(self):
        super().__init__()
        self.tdnn1 = nn.Conv1d(40, 512, kernel_size=5, stride=1, padding=2)
        self.relu = nn.ReLU()
        self.tdnn2 = nn.Conv1d(512, 512, kernel_size=3, stride=1, padding=1)
        self.tdnn3 = nn.Conv1d(512, 512, kernel_size=3, stride=1, padding=1)
        self.tdnn4 = nn.Conv1d(512, 512, kernel_size=1)
        self.tdnn5 = nn.Conv1d(512, 1500, kernel_size=1)
        self.stats_pooling = StatsPooling()
        self.fc1 = nn.Linear(3000, 512)

    def forward(self, x):
        x = self.relu(self.tdnn1(x))
        x = self.relu(self.tdnn2(x))
        x = self.relu(self.tdnn3(x))
        x = self.relu(self.tdnn4(x))
        x = self.relu(self.tdnn5(x))
        x = self.stats_pooling(x)
        x = self.fc1(x)
        return x

modules = {
    "compute_STFT": compute_STFT,
    "compute_fbanks": compute_fbanks,
    "normalize": normalize,
    "embedding_model": FixedXvector(),
    "classifier": Linear(input_size=512, n_neurons=num_languages),
}


hparams = {
    "sample_rate": sample_rate,
    "n_mels": n_mels,
    "compute_cost": torch.nn.functional.cross_entropy,
    "batch_size": 8,
    "train_loader": train_data,
}

In [ ]:
# Класс Brain
class LangIDBrain(sb.Brain):
    def compute_forward(self, batch, stage):
      batch = batch.to(self.device)
      wavs = batch.sig

      wavs_data = wavs.data

      feats = self.modules.compute_STFT(wavs_data)
      feats = self.modules.compute_fbanks(feats)
      feats = self.modules.normalize(feats, wavs.lengths)

      feats = feats.transpose(1, 2)

      # Убираем лишнее измерение, чтобы получить 3D тензор для Conv1d
      feats = feats.mean(dim=-1)  # или feats = feats[..., 0]

      embeddings = self.modules.embedding_model(feats)
      outputs = self.modules.classifier(embeddings)

      return outputs, None

    def compute_objectives(self, predictions, batch, stage):
        outputs, _ = predictions
        labels = batch.lang_encoded
        loss = self.hparams.compute_cost(outputs, labels)
        return loss

    def on_stage_start(self, stage, epoch=None):
        pass

    def on_stage_end(self, stage, stage_loss, epoch=None):
        print(f"{stage.name} loss: {stage_loss:.4f}")

In [ ]:
# ============================
# 6) Training
# ============================
from speechbrain.utils.checkpoints import Checkpointer
from tqdm import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"

brain = LangIDBrain(
  modules=modules,
  hparams=hparams,
  run_opts={"device": device},
  checkpointer=Checkpointer(checkpoints_dir="checkpoints"),

  )

brain.fit(
  epoch_counter=sb.utils.epoch_loop.EpochCounter(limit=10),    #here it's better to change number because 10 is too much
  train_set=train_data,
  valid_set=valid_data,
  train_loader_kwargs={"batch_size": 8, "shuffle": True},
  valid_loader_kwargs={"batch_size": 8},
  )


INFO:speechbrain.core:Gradscaler enabled: `False`
INFO:speechbrain.core:Using training precision: `--precision=fp32`
INFO:speechbrain.core:Using evaluation precision: `--eval_precision=fp32`
INFO:speechbrain.core:LangIDBrain Model Statistics:
* Total Number of Trainable Parameters: 4.2M
* Total Number of Parameters: 4.2M
* Trainable Parameters represent 100.0000% of the total size.
INFO:speechbrain.utils.checkpoints:Would load a checkpoint here, but none found yet.
INFO:speechbrain.utils.epoch_loop:Going into epoch 1
100%|██████████| 952/952 [17:56<00:00,  1.13s/it, train_loss=1.08]


TRAIN loss: 1.0793


100%|██████████| 204/204 [01:39<00:00,  2.04it/s]
INFO:speechbrain.utils.epoch_loop:Going into epoch 2


VALID loss: 1.0793


100%|██████████| 952/952 [17:34<00:00,  1.11s/it, train_loss=1.08]


TRAIN loss: 1.0793


100%|██████████| 204/204 [01:36<00:00,  2.10it/s]
INFO:speechbrain.utils.epoch_loop:Going into epoch 3


VALID loss: 1.0793


 21%|██        | 196/952 [03:54<15:02,  1.19s/it, train_loss=1.08]


KeyboardInterrupt: 

In [ ]:
# ============================
# 7) Testing
# ============================
brain.evaluate(
    test_set=test_data,
    test_loader_kwargs={"batch_size": 8},
)

INFO:speechbrain.utils.checkpoints:Would load a checkpoint here, but none found yet.

  0%|          | 0/204 [00:00<?, ?it/s]


NotImplementedError: 

## evaluation with confusion matrix, error rate, precision, and F1-score

## visualisation wit t-sne